# Metacritic 2021 PC Games Score Scraper

Kallen Hager; 2021-01

I built this scraper intending to answer the following questions for PC games:

* **Do metacritic scores correlate with sales\*?**
    * Should a videogame publisher/developer pay attention to critic reviews?
<BR>
<BR>
* **Which critic's scores correlate best with sales\*?**
    *  Specifically, which critics should a videogame publisher/developer pay attention to?


*Note: I don't actually have sales data. Videogame sales figures are not pubically available. The total amount of reviews a game has on Steam is generally accepted as the best publically available proxy for PC game sales (https://vginsights.com/insights/article/how-to-estimate-steam-video-game-sales).

#### import

In [3]:
#import these libraries, I used Anaconda
import requests                     
from bs4 import BeautifulSoup       #use Python 3.9.7 (Anaconda version)
import pandas as pd
print('pandas version: '+pd.__version__)

pandas version: 1.3.4


### Metacritic Scrape 1
    
&ensp; Using Beatifulsoup, I scraped the `game_name`, `rank`, `meta_score`, `user_score`, and `critic_url` for all 2021 PC games listed on Metacritic into the `games` data frame

##### &emsp; Scraper 1

In [ ]:
games_dict = {'game_name':[], 'rank':[], 'meta_score':[], 'user_score':[], 'critic_url':[]} #create dictionary for desired variables

for page in range(0,4): #each page
    url = 'https://www.metacritic.com/browse/games/score/metascore/year/pc/filtered?year_selected=2021&view=detailed&sort=desc&page='+str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response1 = requests.get(url, headers = user_agent)
    
    # parse response with the specified parser 
    soup = BeautifulSoup(response1.text, 'html.parser')

    # Use the 'inspect element' tool in Chrome on the first Metacritic webpage to find out where to go. This took a good bit of trial an error to figure out which tags I needed to call 
    for temp in soup.find_all('td', class_='clamp-summary-wrap'): #each game summary('clamp-summary-wrap')
        games_dict['game_name'].append(temp.find('a', class_='title').find('h3').text)
        games_dict['rank'].append(temp.find_next('span', class_='title numbered').text.replace("\n","").replace(" ",""))
        games_dict['meta_score'].append(temp.find('div', class_='clamp-metascore').find_next('div').text)
        games_dict['user_score'].append(temp.find('div', class_='clamp-userscore').find_next('div').text)          #^works
        games_dict['critic_url'].append(temp.find('div', class_='clamp-metascore').find_next('a').get('href'))
games = pd.DataFrame(games_dict)
display(games)

### Metacritic Scrape 2

&ensp; Go to each `critic_url` in `games` and scrape `critic_name` and `critic_score` for each critic review into `cricit_df`. Merge dataframes into `game_reviews`, save as `.to_csv`

##### &emsp; Scraper 2

In [258]:
import time
number = 0 # create a number for the counter

critic_dict = {'critic_url':[], 'critic_name':[], 'critic_score':[]}    #create dictionary for desired variables

for each_critic_url in games_dict['critic_url']: #each page             #how to get dictionary list 'critic_url' to loop
    url = 'https://www.metacritic.com'+str(each_critic_url)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response2 = requests.get(url, headers = user_agent)
    
    time.sleep(1)                                                       # fix "Error 429 Slow down", scraper to fast, Metacritic doesn't want to get DDosed or whatever
    
    soup_2 = BeautifulSoup(response2.text, 'html.parser') 
    
    #this writes each page to its own soup_x.html file, is to find errors, some pages produced "Error 429 Slow down" see fix above
    number += 1
    with open(f'soup_2_{number}.html', "w", encoding="utf-8") as file: 
        file.write(str(soup_2))
    
    # I figured out at least one game didn't actually have a review page
    if soup_2.find('title').text == '404 Page Not Found - Metacritic - Metacritic':
        critic_dict['critic_url'].append('null_404') 
        critic_dict['critic_name'].append('null_404') 
        critic_dict['critic_score'].append('null_404')
        continue
    
    # again, a lot of trial and error to figure out which things I needed to pull
    # for class_='string' , the string specificed only has to match the first part of html class. EX: class_='example' will pull 'example' and 'example turtle' and 'example ham'    
    for temp in soup_2.find('div', class_='body product_reviews').find_next('ol', class_='reviews critic_reviews').find_all('li', class_= 'review'):
        critic_dict['critic_url'].append(each_critic_url) 
        critic_dict['critic_name'].append(temp.find_next('div', class_='source').text)
        critic_dict['critic_score'].append(temp.find_next('div', class_='review_grade').find_next('div').text)
critic_df = pd.DataFrame(critic_dict)
display(critic_df)

,critic_url,critic_name,critic_score
0,/game/pc/disco-elysium-the-final-cut/critic-re...,GameGrin,100
1,/game/pc/disco-elysium-the-final-cut/critic-re...,Shindig,100
2,/game/pc/disco-elysium-the-final-cut/critic-re...,KeenGamer,100
3,/game/pc/disco-elysium-the-final-cut/critic-re...,GameSpot,100
4,/game/pc/disco-elysium-the-final-cut/critic-re...,GameSpot,100
...,...,...,...
5783,/game/pc/efootball-2022/critic-reviews,GameStar,28
5784,/game/pc/efootball-2022/critic-reviews,GameStar,28
5785,/game/pc/efootball-2022/critic-reviews,Gamer.no,20
5786,/game/pc/efootball-2022/critic-reviews,PC Games,20


In [259]:
#save to_csv()
critic_df.to_csv('game_reviews.csv', index=False)

##### &emsp; Debugging Tools for Scraper 2:

In [ ]:
# this block correctly prints each url
for each_critic_url in review_dict['critic_url']: #each page
    url = 'https://www.metacritic.com'+str(each_critic_url)
    print(url) 

In [ ]:
# write soup_2 to a file so I don't have to scrape it each time
# modified to write soup_2_# to check for error, some files retrieve "Error 429 Slow down" instead, added time.sleep(1) to fix
critic_dict = {'critic_url':[], 'critic_name':[], 'critic_score':[]} #create dictionary for desired variables
number = 0
for each_critic_url in review_dict['critic_url']: #each page                              #how to get dictionary list 'critic_url' to loop
    url = 'https://www.metacritic.com'+str(each_critic_url)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers = user_agent)

    time.sleep(1)
    
    soup_2 = BeautifulSoup(response.text, 'html.parser')

    number += 1
    with open(f'soup_2_{number}.html', "w", encoding="utf-8") as file:
        file.write(str(soup_2))

In [ ]:
#reads in soup_2.html
with open("soup_2.html") as fp:
    soup_2 = BeautifulSoup(fp, 'html.parser')
display(soup_2)

In [ ]:
# Getting too many entries returned for each dictionary key
# critic_name returning 'Read full review'
critic_dict = {'critic_url':[], 'critic_name':[], 'critic_score':[]} #create dictionary for desired variables
for temp in soup_2.find('div', class_='body product_reviews').find_next('ol', class_='reviews critic_reviews').find_all('li', class_= 'review'):
    critic_dict['critic_url'].append(each_critic_url) 
    critic_dict['critic_name'].append(temp.find_next('div', class_='source').text)
    critic_dict['critic_score'].append(temp.find_next('div', class_='review_grade').find_next('div').text)
print(critic_dict)

In [ ]:
# save 'temp_{number}' file for each 'li' found in soup_2
number = 0
for temp in soup_2.find('div', class_='body product_reviews').find_next('ol', class_='reviews critic_reviews').find_all('li', class_= 'review'):
    number = number + 1
    with open(f'temp_{number}', 'w') as file:
        file.write(str(temp))

##### &emsp; Merge and Save

In [260]:
# drop_duplicates()
dropped_df = critic_df.drop_duplicates()
display(dropped_df)

,critic_url,critic_name,critic_score
0,/game/pc/disco-elysium-the-final-cut/critic-re...,GameGrin,100
1,/game/pc/disco-elysium-the-final-cut/critic-re...,Shindig,100
2,/game/pc/disco-elysium-the-final-cut/critic-re...,KeenGamer,100
3,/game/pc/disco-elysium-the-final-cut/critic-re...,GameSpot,100
5,/game/pc/disco-elysium-the-final-cut/critic-re...,Wccftech,100
...,...,...,...
5782,/game/pc/efootball-2022/critic-reviews,PC Gamer,30
5783,/game/pc/efootball-2022/critic-reviews,GameStar,28
5785,/game/pc/efootball-2022/critic-reviews,Gamer.no,20
5786,/game/pc/efootball-2022/critic-reviews,PC Games,20


In [261]:
# pivot() the critic_df dataframe
pivoted_df = dropped_df.pivot(index='critic_url', columns='critic_name', values='critic_score')
display(pivoted_df)

critic_name,4Players.de,Adventure Gamers,Android Central,App Trigger,Areajugones,Atomix,Attack of the Fanboy,AusGamers,BaziCenter,Buried Treasure,...,Vgames,Washington Post,Wccftech,We Got This Covered,WellPlayed,Windows Central,Worth Playing,XGN,ZTGD,null_404
critic_url,,,,,,,,,,,,,,,,,,,,,
/game/pc/adios/critic-reviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/game/pc/aeterna-noctis/critic-reviews,NaN,NaN,NaN,NaN,70,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,70,NaN,NaN,NaN
/game/pc/after-the-fall/critic-reviews,NaN,NaN,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,70,NaN,NaN,NaN
/game/pc/age-of-empires-iv/critic-reviews,80,NaN,NaN,NaN,90,90,NaN,95,NaN,NaN,...,NaN,NaN,90,NaN,70,100,79,75,90,NaN
/game/pc/alex-kidd-in-miracle-world-dx/critic-reviews,NaN,NaN,NaN,NaN,75,80,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/game/pc/wrc-10-fia-world-rally-championship/critic-reviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/game/pc/wytchwood/critic-reviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/game/pc/yakuza-6-the-song-of-life/critic-reviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [262]:
# merge() pivoted critic_df left into games dataframe
game_reviews = games.merge(pivoted_df, on='critic_url', how='left')
display(game_reviews)

,game_name,rank,meta_score,user_score,critic_url,4Players.de,Adventure Gamers,Android Central,App Trigger,Areajugones,...,Vgames,Washington Post,Wccftech,We Got This Covered,WellPlayed,Windows Central,Worth Playing,XGN,ZTGD,null_404
0,Disco Elysium: The Final Cut,1.,97,8.2,/game/pc/disco-elysium-the-final-cut/critic-re...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Final Fantasy XIV: Endwalker,2.,92,9.5,/game/pc/final-fantasy-xiv-endwalker/critic-re...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,90,NaN,NaN,NaN,NaN
2,Forza Horizon 5,3.,91,6.9,/game/pc/forza-horizon-5/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,95,NaN,85,NaN,NaN,NaN,NaN,NaN
3,Chicory: A Colorful Tale,4.,90,7.4,/game/pc/chicory-a-colorful-tale/critic-reviews,NaN,100,NaN,NaN,NaN,...,NaN,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Psychonauts 2,5.,89,7.0,/game/pc/psychonauts-2/critic-reviews,NaN,NaN,NaN,NaN,82,...,NaN,NaN,95,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,Demon Skin,316.,48,7.4,/game/pc/demon-skin/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,35,NaN,70,NaN,NaN,NaN,NaN,NaN
316,Arkham Horror: Mother's Embrace,317.,48,6.0,/game/pc/arkham-horror-mothers-embrace/critic-...,39,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
317,Of Bird and Cage,318.,44,7.0,/game/pc/of-bird-and-cage/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
318,Balan Wonderworld,319.,38,6.5,/game/pc/balan-wonderworld/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [263]:
#save to_csv()
game_reviews.to_csv('game_reviews.csv', index=False)

In [ ]:
#read_csv and display
game_reviews = pd.read_csv('game_reviews.csv')
display(game_reviews)

### Steam API 1

&ensp; Get Steam `app_id` from the Steam API, merge with `game_reviews` dataframe

##### &emsp; API Pull

In [18]:
# Get all appid and name from Steam's API
api_key = 'D21B7B11D12663825753205E190103C4'                                   # you can sign up for a key here https://steamcommunity.com/dev (use ur github domain) 
url = f'https://api.steampowered.com/ISteamApps/GetAppList/v2/?key={api_key}'  # url from here https://partner.steamgames.com/doc/webapi/ISteamApps 
user_agent = {'User-agent': 'Mozilla/5.0'}
response3 = requests.get(url, headers = user_agent).json() #get .json object

# Steam API puts a dictionary 'applist' with key value pair 'apps' inside eachother. Then it takes dictionary 'apps' with key value pair of a list. Hence, response is a list.
response3 = response3.get('applist').get('apps')
display(response3)

[{'appid': 1828290, 'name': 'The Glitch Fairy Beta Demo'},
 {'appid': 493900, 'name': 'Dungeons 3'},
 {'appid': 1771380, 'name': 'The Hidden Room'},
 {'appid': 1828741, 'name': ''},
 {'appid': 216938, 'name': 'Pieterw test app76 ( 216938 )'},
 {'appid': 660010, 'name': 'test2'},
 {'appid': 660130, 'name': 'test3'},
 {'appid': 1118314, 'name': ''},
 {'appid': 1275822, 'name': ''},
 {'appid': 1343832, 'name': ''},
 {'appid': 662172, 'name': ''},
 {'appid': 1360782, 'name': ''},
 {'appid': 1063730, 'name': 'New World'},
 {'appid': 1705150, 'name': 'SKULL CHAINZ'},
 {'appid': 1205950, 'name': 'Seeds of Chaos'},
 {'appid': 856890, 'name': 'JUSTICE SUCKS: RECHARGED'},
 {'appid': 1478220, 'name': 'Puzzle Together'},
 {'appid': 847490, 'name': 'Star Speeder'},
 {'appid': 1238880, 'name': 'Battlefield™ Hardline'},
 {'appid': 1521970, 'name': 'Goblin Stone'},
 {'appid': 1578790, 'name': 'Goblin Stone Demo'},
 {'appid': 1519420, 'name': 'DMN7'},
 {'appid': 1565060, 'name': 'Revenants'},
 {'appid'

In [4]:
# Turn the json list `response3` into a dataframe `appid_names_df`
appid_name = pd.DataFrame(response3)    
display(appid_name)
display(len(appid_name))

,appid,name
0,1446310,Super/Human Identity
1,1446320,Harvest Island: Demo
2,1446330,100 hidden snails
3,1446350,You Will Die Here Tonight
4,1446360,Age of Empires III: Definitive Edition Soundtrack
...,...,...
135132,1866910,Time Stop - The Female Knight
135133,1376580,Alien Dawn
135134,1301410,Solar Rogue
135135,413150,Stardew Valley


135137

##### &emsp;Merge

In [294]:
#visualize what needs to be merged
game_reviews = pd.read_csv('game_reviews.csv') # read_csv saved earlier
display(game_reviews)
display(appid_name)

,game_name,rank,meta_score,user_score,critic_url,4Players.de,Adventure Gamers,Android Central,App Trigger,Areajugones,...,Vgames,Washington Post,Wccftech,We Got This Covered,WellPlayed,Windows Central,Worth Playing,XGN,ZTGD,null_404
0,Disco Elysium: The Final Cut,1.0,97,8.2,/game/pc/disco-elysium-the-final-cut/critic-re...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Final Fantasy XIV: Endwalker,2.0,92,9.5,/game/pc/final-fantasy-xiv-endwalker/critic-re...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,90.0,NaN,NaN,NaN,NaN
2,Forza Horizon 5,3.0,91,6.9,/game/pc/forza-horizon-5/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,95.0,NaN,85.0,NaN,NaN,NaN,NaN,NaN
3,Chicory: A Colorful Tale,4.0,90,7.4,/game/pc/chicory-a-colorful-tale/critic-reviews,NaN,100.0,NaN,NaN,NaN,...,NaN,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Psychonauts 2,5.0,89,7.0,/game/pc/psychonauts-2/critic-reviews,NaN,NaN,NaN,NaN,82.0,...,NaN,NaN,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,Demon Skin,316.0,48,7.4,/game/pc/demon-skin/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,35.0,NaN,70.0,NaN,NaN,NaN,NaN,NaN
316,Arkham Horror: Mother's Embrace,317.0,48,6.0,/game/pc/arkham-horror-mothers-embrace/critic-...,39.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
317,Of Bird and Cage,318.0,44,7.0,/game/pc/of-bird-and-cage/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
318,Balan Wonderworld,319.0,38,6.5,/game/pc/balan-wonderworld/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,appid,name
0,216938,Pieterw test app76 ( 216938 )
1,660010,test2
2,660130,test3
3,1118314,
4,1275822,
...,...,...
135003,1863090,Word Crack
135004,1836290,They Die Tomorrow
135005,258550,Rust Dedicated Server
135006,1307710,GRID Legends


In [ ]:
# for loops to get rid of some special characters that differed in Metacritic's game_name and Steam's name
game_reviews['parsed_name']=[s.replace(':', '').replace('-', '').replace(' ', '').replace('™', '').replace("'", '')  for s in game_reviews['game_name']] 
appid_name['parsed_name']=[s.replace(':', '').replace('-', '').replace(' ', '').replace('™', '').replace("'", '') for s in appid_name['name']]
display(game_reviews)
display(appid_name)    

,game_name,rank,meta_score,user_score,critic_url,4Players.de,Adventure Gamers,Android Central,App Trigger,Areajugones,...,Washington Post,Wccftech,We Got This Covered,WellPlayed,Windows Central,Worth Playing,XGN,ZTGD,null_404,parsed_name
0,Disco Elysium: The Final Cut,1.0,97,8.2,/game/pc/disco-elysium-the-final-cut/critic-re...,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DiscoElysiumTheFinalCut
1,Final Fantasy XIV: Endwalker,2.0,92,9.5,/game/pc/final-fantasy-xiv-endwalker/critic-re...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,90.0,NaN,NaN,NaN,NaN,FinalFantasyXIVEndwalker
2,Forza Horizon 5,3.0,91,6.9,/game/pc/forza-horizon-5/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,NaN,95.0,NaN,85.0,NaN,NaN,NaN,NaN,NaN,ForzaHorizon5
3,Chicory: A Colorful Tale,4.0,90,7.4,/game/pc/chicory-a-colorful-tale/critic-reviews,NaN,100.0,NaN,NaN,NaN,...,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ChicoryAColorfulTale
4,Psychonauts 2,5.0,89,7.0,/game/pc/psychonauts-2/critic-reviews,NaN,NaN,NaN,NaN,82.0,...,NaN,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Psychonauts2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,Demon Skin,316.0,48,7.4,/game/pc/demon-skin/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,NaN,35.0,NaN,70.0,NaN,NaN,NaN,NaN,NaN,DemonSkin
316,Arkham Horror: Mother's Embrace,317.0,48,6.0,/game/pc/arkham-horror-mothers-embrace/critic-...,39.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ArkhamHorrorMothersEmbrace
317,Of Bird and Cage,318.0,44,7.0,/game/pc/of-bird-and-cage/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OfBirdandCage
318,Balan Wonderworld,319.0,38,6.5,/game/pc/balan-wonderworld/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BalanWonderworld


,appid,name,parsed_name
0,216938,Pieterw test app76 ( 216938 ),Pieterwtestapp76(216938)
1,660010,test2,test2
2,660130,test3,test3
3,1118314,,
4,1275822,,
...,...,...,...
135003,1863090,Word Crack,WordCrack
135004,1836290,They Die Tomorrow,TheyDieTomorrow
135005,258550,Rust Dedicated Server,RustDedicatedServer
135006,1307710,GRID Legends,GRIDLegends


In [325]:
# save game_reviews as parsed_game_reviews.csv  
game_reviews.to_csv('parsed_game_reviews.csv', index=False)

In [326]:
# read parsed_game_reviews.csv as game_reviews 
game_reviews = pd.read_csv('parsed_game_reviews.csv')
display(game_reviews)

,game_name,rank,meta_score,user_score,critic_url,4Players.de,Adventure Gamers,Android Central,App Trigger,Areajugones,...,Washington Post,Wccftech,We Got This Covered,WellPlayed,Windows Central,Worth Playing,XGN,ZTGD,null_404,parsed_name
0,Disco Elysium: The Final Cut,1.0,97,8.2,/game/pc/disco-elysium-the-final-cut/critic-re...,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DiscoElysiumTheFinalCut
1,Final Fantasy XIV: Endwalker,2.0,92,9.5,/game/pc/final-fantasy-xiv-endwalker/critic-re...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,90.0,NaN,NaN,NaN,NaN,FinalFantasyXIVEndwalker
2,Forza Horizon 5,3.0,91,6.9,/game/pc/forza-horizon-5/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,NaN,95.0,NaN,85.0,NaN,NaN,NaN,NaN,NaN,ForzaHorizon5
3,Chicory: A Colorful Tale,4.0,90,7.4,/game/pc/chicory-a-colorful-tale/critic-reviews,NaN,100.0,NaN,NaN,NaN,...,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ChicoryAColorfulTale
4,Psychonauts 2,5.0,89,7.0,/game/pc/psychonauts-2/critic-reviews,NaN,NaN,NaN,NaN,82.0,...,NaN,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Psychonauts2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,Demon Skin,316.0,48,7.4,/game/pc/demon-skin/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,NaN,35.0,NaN,70.0,NaN,NaN,NaN,NaN,NaN,DemonSkin
316,Arkham Horror: Mother's Embrace,317.0,48,6.0,/game/pc/arkham-horror-mothers-embrace/critic-...,39.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ArkhamHorrorMothersEmbrace
317,Of Bird and Cage,318.0,44,7.0,/game/pc/of-bird-and-cage/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OfBirdandCage
318,Balan Wonderworld,319.0,38,6.5,/game/pc/balan-wonderworld/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BalanWonderworld


In [327]:
dropped_df2 = game_reviews.drop_duplicates()
display(dropped_df2)

,game_name,rank,meta_score,user_score,critic_url,4Players.de,Adventure Gamers,Android Central,App Trigger,Areajugones,...,Washington Post,Wccftech,We Got This Covered,WellPlayed,Windows Central,Worth Playing,XGN,ZTGD,null_404,parsed_name
0,Disco Elysium: The Final Cut,1.0,97,8.2,/game/pc/disco-elysium-the-final-cut/critic-re...,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DiscoElysiumTheFinalCut
1,Final Fantasy XIV: Endwalker,2.0,92,9.5,/game/pc/final-fantasy-xiv-endwalker/critic-re...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,90.0,NaN,NaN,NaN,NaN,FinalFantasyXIVEndwalker
2,Forza Horizon 5,3.0,91,6.9,/game/pc/forza-horizon-5/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,NaN,95.0,NaN,85.0,NaN,NaN,NaN,NaN,NaN,ForzaHorizon5
3,Chicory: A Colorful Tale,4.0,90,7.4,/game/pc/chicory-a-colorful-tale/critic-reviews,NaN,100.0,NaN,NaN,NaN,...,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ChicoryAColorfulTale
4,Psychonauts 2,5.0,89,7.0,/game/pc/psychonauts-2/critic-reviews,NaN,NaN,NaN,NaN,82.0,...,NaN,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Psychonauts2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,Demon Skin,316.0,48,7.4,/game/pc/demon-skin/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,NaN,35.0,NaN,70.0,NaN,NaN,NaN,NaN,NaN,DemonSkin
316,Arkham Horror: Mother's Embrace,317.0,48,6.0,/game/pc/arkham-horror-mothers-embrace/critic-...,39.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ArkhamHorrorMothersEmbrace
317,Of Bird and Cage,318.0,44,7.0,/game/pc/of-bird-and-cage/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OfBirdandCage
318,Balan Wonderworld,319.0,38,6.5,/game/pc/balan-wonderworld/critic-reviews,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BalanWonderworld


In [328]:
#merge the dataframes                                                                                    casefold() so the cases are lowercase and match 
appid_reviews = pd.merge(game_reviews, appid_name, how = 'left', left_on=game_reviews['parsed_name'].str.casefold(), right_on=appid_name['parsed_name'].str.casefold())
#appid_reviews = pd.merge(game_reviews, appid_name, on=game_reviews['parsed_name'].str.casefold(), how="left")

#display new df
display(appid_reviews)

#print out each column name
cols = list(appid_reviews.columns.values)
print(cols)

,key_0,game_name,rank,meta_score,user_score,critic_url,4Players.de,Adventure Gamers,Android Central,App Trigger,...,WellPlayed,Windows Central,Worth Playing,XGN,ZTGD,null_404,parsed_name_x,appid,name,parsed_name_y
0,discoelysiumthefinalcut,Disco Elysium: The Final Cut,1.0,97,8.2,/game/pc/disco-elysium-the-final-cut/critic-re...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,DiscoElysiumTheFinalCut,NaN,NaN,NaN
1,finalfantasyxivendwalker,Final Fantasy XIV: Endwalker,2.0,92,9.5,/game/pc/final-fantasy-xiv-endwalker/critic-re...,NaN,NaN,NaN,NaN,...,NaN,90.0,NaN,NaN,NaN,NaN,FinalFantasyXIVEndwalker,NaN,NaN,NaN
2,forzahorizon5,Forza Horizon 5,3.0,91,6.9,/game/pc/forza-horizon-5/critic-reviews,NaN,NaN,NaN,NaN,...,85.0,NaN,NaN,NaN,NaN,NaN,ForzaHorizon5,1551360.0,Forza Horizon 5,ForzaHorizon5
3,chicoryacolorfultale,Chicory: A Colorful Tale,4.0,90,7.4,/game/pc/chicory-a-colorful-tale/critic-reviews,NaN,100.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ChicoryAColorfulTale,1123450.0,Chicory: A Colorful Tale,ChicoryAColorfulTale
4,psychonauts2,Psychonauts 2,5.0,89,7.0,/game/pc/psychonauts-2/critic-reviews,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Psychonauts2,607080.0,Psychonauts 2,Psychonauts2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,demonskin,Demon Skin,316.0,48,7.4,/game/pc/demon-skin/critic-reviews,NaN,NaN,NaN,NaN,...,70.0,NaN,NaN,NaN,NaN,NaN,DemonSkin,1522140.0,Demon Skin,DemonSkin
321,arkhamhorrormothersembrace,Arkham Horror: Mother's Embrace,317.0,48,6.0,/game/pc/arkham-horror-mothers-embrace/critic-...,39.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ArkhamHorrorMothersEmbrace,840210.0,Arkham Horror: Mother's Embrace,ArkhamHorrorMothersEmbrace
322,ofbirdandcage,Of Bird and Cage,318.0,44,7.0,/game/pc/of-bird-and-cage/critic-reviews,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,OfBirdandCage,523770.0,Of Bird And Cage,OfBirdAndCage
323,balanwonderworld,Balan Wonderworld,319.0,38,6.5,/game/pc/balan-wonderworld/critic-reviews,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,BalanWonderworld,1341050.0,BALAN WONDERWORLD,BALANWONDERWORLD


['key_0', 'game_name', 'rank', 'meta_score', 'user_score', 'critic_url', '4Players.de', 'Adventure Gamers', 'Android Central', 'App Trigger', 'Areajugones', 'Atomix', 'Attack of the Fanboy', 'AusGamers', 'BaziCenter', 'Buried Treasure', 'But Why Tho?', 'CD-Action', 'CGMagazine', 'COGconnected', 'Carole Quintaine', 'Checkpoint Gaming', 'Comicbook.com', 'Critical Hit', 'Cubed3', 'Cultured Vultures', 'DarkStation', 'DarkZero', 'Destructoid', 'Dexerto', 'Digital Chumps', 'Digital Spy', 'Digital Trends', 'Digitally Downloaded', 'DualShockers', 'EGM', 'Easy Allies', 'Edge Magazine', 'Eurogamer Italy', 'Everyeye.it', 'Finger Guns', 'Forbes', 'GAMES.CH', 'GAMINGbible', 'GMW3', 'GRYOnline.pl', 'Game Debate', 'Game Informer', 'Game Over Online', 'Game Rant', 'Game Revolution', 'Game World Navigator Magazine', 'GameByte', 'GameCrate', 'GameCritics', 'GameGrin', 'GameMAG', 'GameOver.gr', 'GameSkinny', 'GameSpace', 'GameSpew', 'GameSpot', 'GameStar', 'GameWatcher', 'Gameblog.fr', 'Gamepressure', 'G

In [330]:
#rename columns and move to front
col = appid_reviews.pop('appid')
appid_reviews.insert(0, 'appid', col)

col2 = appid_reviews.pop('name')
appid_reviews.insert(1, 'name', col2)

# drop extra columns created by merge
#appid_reviews.drop(appid_reviews.columns[[-1, -2]], axis=1, inplace = True)                  # drop by column number (parsed_name_x and parsed_name_y)
#appid_reviews.drop('key_0', axis=1, inplace=True)                                            # drop by column name
appid_reviews.drop_duplicates(inplace=True)
display(appid_reviews)

,appid,name,key_0,game_name,rank,meta_score,user_score,critic_url,4Players.de,Adventure Gamers,...,Wccftech,We Got This Covered,WellPlayed,Windows Central,Worth Playing,XGN,ZTGD,null_404,parsed_name_x,parsed_name_y
0,NaN,NaN,discoelysiumthefinalcut,Disco Elysium: The Final Cut,1.0,97,8.2,/game/pc/disco-elysium-the-final-cut/critic-re...,NaN,NaN,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DiscoElysiumTheFinalCut,NaN
1,NaN,NaN,finalfantasyxivendwalker,Final Fantasy XIV: Endwalker,2.0,92,9.5,/game/pc/final-fantasy-xiv-endwalker/critic-re...,NaN,NaN,...,NaN,NaN,NaN,90.0,NaN,NaN,NaN,NaN,FinalFantasyXIVEndwalker,NaN
2,1551360.0,Forza Horizon 5,forzahorizon5,Forza Horizon 5,3.0,91,6.9,/game/pc/forza-horizon-5/critic-reviews,NaN,NaN,...,95.0,NaN,85.0,NaN,NaN,NaN,NaN,NaN,ForzaHorizon5,ForzaHorizon5
3,1123450.0,Chicory: A Colorful Tale,chicoryacolorfultale,Chicory: A Colorful Tale,4.0,90,7.4,/game/pc/chicory-a-colorful-tale/critic-reviews,NaN,100.0,...,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ChicoryAColorfulTale,ChicoryAColorfulTale
4,607080.0,Psychonauts 2,psychonauts2,Psychonauts 2,5.0,89,7.0,/game/pc/psychonauts-2/critic-reviews,NaN,NaN,...,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Psychonauts2,Psychonauts2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,1522140.0,Demon Skin,demonskin,Demon Skin,316.0,48,7.4,/game/pc/demon-skin/critic-reviews,NaN,NaN,...,35.0,NaN,70.0,NaN,NaN,NaN,NaN,NaN,DemonSkin,DemonSkin
321,840210.0,Arkham Horror: Mother's Embrace,arkhamhorrormothersembrace,Arkham Horror: Mother's Embrace,317.0,48,6.0,/game/pc/arkham-horror-mothers-embrace/critic-...,39.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ArkhamHorrorMothersEmbrace,ArkhamHorrorMothersEmbrace
322,523770.0,Of Bird And Cage,ofbirdandcage,Of Bird and Cage,318.0,44,7.0,/game/pc/of-bird-and-cage/critic-reviews,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OfBirdandCage,OfBirdAndCage
323,1341050.0,BALAN WONDERWORLD,balanwonderworld,Balan Wonderworld,319.0,38,6.5,/game/pc/balan-wonderworld/critic-reviews,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BalanWonderworld,BALANWONDERWORLD


In [336]:
# noticed the new dataframe has 5 extra rows created during the merge
# check for duplicates by column `game_name`
duplicate_df = appid_reviews[appid_reviews.duplicated(subset=['game_name'], keep=False)]
display(duplicate_df)

,appid,name,key_0,game_name,rank,meta_score,user_score,critic_url,4Players.de,Adventure Gamers,...,Wccftech,We Got This Covered,WellPlayed,Windows Central,Worth Playing,XGN,ZTGD,null_404,parsed_name_x,parsed_name_y
229,844740.0,SCARF,scarf,Scarf,230.0,70,8.3,/game/pc/scarf/critic-reviews,NaN,NaN,...,NaN,NaN,70.0,NaN,NaN,NaN,NaN,NaN,Scarf,SCARF
230,645320.0,SCARF,scarf,Scarf,230.0,70,8.3,/game/pc/scarf/critic-reviews,NaN,NaN,...,NaN,NaN,70.0,NaN,NaN,NaN,NaN,NaN,Scarf,SCARF
263,506440.0,Nerve,nerve,Nerve,263.0,67,tbd,/game/pc/nerve/critic-reviews,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nerve,Nerve
264,1016000.0,Nerve,nerve,Nerve,263.0,67,tbd,/game/pc/nerve/critic-reviews,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nerve,Nerve
294,1452500.0,The Good Life,thegoodlife,The Good Life,293.0,62,5.7,/game/pc/the-good-life/critic-reviews,NaN,NaN,...,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TheGoodLife,TheGoodLife
295,293340.0,The Good Life,thegoodlife,The Good Life,293.0,62,5.7,/game/pc/the-good-life/critic-reviews,NaN,NaN,...,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TheGoodLife,TheGoodLife
309,752030.0,Paradise Lost,paradiselost,Paradise Lost,307.0,58,6.8,/game/pc/paradise-lost/critic-reviews,52.0,70.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ParadiseLost,ParadiseLost
310,1555890.0,Paradise Lost,paradiselost,Paradise Lost,307.0,58,6.8,/game/pc/paradise-lost/critic-reviews,52.0,70.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ParadiseLost,ParadiseLost
311,982720.0,Paradise Lost,paradiselost,Paradise Lost,307.0,58,6.8,/game/pc/paradise-lost/critic-reviews,52.0,70.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ParadiseLost,ParadiseLost


In [338]:
# a few game_names from Metacritic have multiple name matches on Steam, I  will go check Metacritic and Steam to see which name/appid is correct
# drop the rows by index
appid_reviews.drop([229, 263, 295, 309, 310],inplace=True)

# check that we are back to the right number for rows (320)
display(len(appid_reviews))

320

In [339]:
# save dataframe to_csv
appid_reviews.to_csv('appid_reviews.csv', index=False)

### Steam API 2

&ensp; Use `app_id` to get `total_reviews` from the Steam API  

##### &emsp;Checking appid column

In [ ]:
# pull up missing values in column appid
appid_reviews = pd.read_csv('appid_reviews.csv')
appid_reviews.loc[appid_reviews['appid'].isna()]

There are 54 game_names that did not match to an appid. I opened the CSV in Excel and manually added the appid's that were available and valid. I entered 'delete' for games that are not on Steam, or games that are not new 2021 releases (complete editions, remakes, expansion, collections, episodic etc.).

In [39]:
# read in manually modified csv 
appid_reviews_mod = pd.read_csv('appid_reviews_mod.csv')
display(appid_reviews_mod) # check number of rows to make sure everything is good
appid_reviews_mod.loc[appid_reviews_mod['appid'].isna()] # make sure all NaN in 'appid' are gone now

,appid,name,key_0,game_name,rank,meta_score,user_score,critic_url,4Players.de,Adventure Gamers,...,Wccftech,We Got This Covered,WellPlayed,Windows Central,Worth Playing,XGN,ZTGD,null_404,parsed_name_x,parsed_name_y
0,1824220,Chivalry 2,chivalry2,Chivalry 2,45,82,5,/game/pc/chivalry-2/critic-reviews,83.0,NaN,...,NaN,NaN,NaN,90.0,83.0,NaN,NaN,NaN,Chivalry2,Chivalry2
1,1716120,Monster Rancher 1 & 2 DX,monsterrancher1&2dx,Monster Rancher 1 & 2 DX,294,61,7.5,/game/pc/monster-rancher-1-2-dx/critic-reviews,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MonsterRancher1&2DX,MonsterRancher1&2DX
2,1665460,eFootball™ 2022,efootball2022,eFootball 2022,320,25,1.4,/game/pc/efootball-2022/critic-reviews,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eFootball2022,eFootball2022
3,1659040,HITMAN 3,hitman3,Hitman 3,8,87,7.4,/game/pc/hitman-3/critic-reviews,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,85.0,NaN,Hitman3,HITMAN3
4,1651960,Hextech Mayhem: A League of Legends Story™,hextechmayhemaleagueoflegendsstory,Hextech Mayhem: A League of Legends Story,206,72,7.3,/game/pc/hextech-mayhem-a-league-of-legends-st...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HextechMayhemALeagueofLegendsStory,HextechMayhemALeagueofLegendsStory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,delete,NaN,jettthefarshore,JETT: The Far Shore,272,66,6.1,/game/pc/jett-the-far-shore/critic-reviews,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JETTTheFarShore,NaN
316,delete,NaN,immortalsfenyxrisingmythsoftheeasternrealm,Immortals Fenyx Rising: Myths of the Eastern R...,273,65,5.8,/game/pc/immortals-fenyx-rising-myths-of-the-e...,NaN,NaN,...,60.0,NaN,NaN,NaN,70.0,NaN,NaN,NaN,ImmortalsFenyxRisingMythsoftheEasternRealm,NaN
317,1273690,NaN,ashwalkersasurvivaljourney,Ashwalkers: A Survival Journey,298,61,tbd,/game/pc/ashwalkers-a-survival-journey/critic-...,36.0,NaN,...,NaN,NaN,NaN,NaN,80.0,NaN,75.0,NaN,AshwalkersASurvivalJourney,NaN
318,delete,NaN,cyanide&happinessfreakpocalypse,Cyanide & Happiness - Freakpocalypse,300,60,tbd,/game/pc/cyanide-happiness---freakpocalypse/cr...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cyanide&HappinessFreakpocalypse,NaN


,appid,name,key_0,game_name,rank,meta_score,user_score,critic_url,4Players.de,Adventure Gamers,...,Wccftech,We Got This Covered,WellPlayed,Windows Central,Worth Playing,XGN,ZTGD,null_404,parsed_name_x,parsed_name_y


In [73]:
# select all rows where 'appid' != 'delete'
appid_reviews_mod = appid_reviews_mod[appid_reviews_mod.appid != 'delete']

#save to_csv()
appid_reviews_mod.to_csv('appid_reviews_mod.csv', index=False)

display(appid_reviews_mod)

,appid,name,key_0,game_name,rank,meta_score,user_score,critic_url,4Players.de,Adventure Gamers,...,Wccftech,We Got This Covered,WellPlayed,Windows Central,Worth Playing,XGN,ZTGD,null_404,parsed_name_x,parsed_name_y
0,1824220,Chivalry 2,chivalry2,Chivalry 2,45,82,5,/game/pc/chivalry-2/critic-reviews,83.0,NaN,...,NaN,NaN,NaN,90.0,83.0,NaN,NaN,NaN,Chivalry2,Chivalry2
1,1716120,Monster Rancher 1 & 2 DX,monsterrancher1&2dx,Monster Rancher 1 & 2 DX,294,61,7.5,/game/pc/monster-rancher-1-2-dx/critic-reviews,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MonsterRancher1&2DX,MonsterRancher1&2DX
2,1665460,eFootball™ 2022,efootball2022,eFootball 2022,320,25,1.4,/game/pc/efootball-2022/critic-reviews,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eFootball2022,eFootball2022
3,1659040,HITMAN 3,hitman3,Hitman 3,8,87,7.4,/game/pc/hitman-3/critic-reviews,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,85.0,NaN,Hitman3,HITMAN3
4,1651960,Hextech Mayhem: A League of Legends Story™,hextechmayhemaleagueoflegendsstory,Hextech Mayhem: A League of Legends Story,206,72,7.3,/game/pc/hextech-mayhem-a-league-of-legends-st...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HextechMayhemALeagueofLegendsStory,HextechMayhemALeagueofLegendsStory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,1623390,NaN,unplugged(vertigogames),Unplugged (Vertigo Games),179,74,tbd,/game/pc/unplugged-vertigo-games/critic-reviews,NaN,NaN,...,NaN,NaN,NaN,NaN,67.0,NaN,NaN,NaN,Unplugged(VertigoGames),NaN
305,921800,NaN,encasedascifipostapocalypticrpg,Encased: a sci-fi post-apocalyptic RPG,195,73,7.7,/game/pc/encased-a-sci-fi-post-apocalyptic-rpg...,NaN,NaN,...,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EncasedascifipostapocalypticRPG,NaN
311,1732190,NaN,fatalframemaidenofblackwater,Fatal Frame: Maiden of Black Water,236,69,5,/game/pc/fatal-frame-maiden-of-black-water/cri...,NaN,NaN,...,60.0,NaN,NaN,70.0,NaN,NaN,NaN,NaN,FatalFrameMaidenofBlackWater,NaN
313,962700,NaN,chronicleofinnsmouthmountainsofmadness,Chronicle of Innsmouth: Mountains of Madness,252,68,7.6,/game/pc/chronicle-of-innsmouth-mountains-of-m...,NaN,60.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ChronicleofInnsmouthMountainsofMadness,NaN


##### &emsp; API Pull 2

In [ ]:
appid_list = appid_reviews_mod['appid'].tolist()
print(appid_list) # sometimes print() is prettier

In [59]:
steam_reviews_dict = {'appid':[], 'total_reviews':[], 'pos_reviews':[], 'neg_reviews':[]}

api_key = 'D21B7B11D12663825753205E190103C4' 
for each_appid in appid_list:
    url = f'https://store.steampowered.com/appreviews/{each_appid}?json=1&language=all&purchase_type=all&?key={api_key}'    
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response4 = requests.get(url)
    response4.encoding = 'utf-8-sig' # fix encoding error
    response4 = response4.json()

    response4 = response4.get('query_summary')
    steam_reviews_dict['appid'].append(each_appid)
    steam_reviews_dict['total_reviews'].append(response4.get('total_reviews'))
    steam_reviews_dict['pos_reviews'].append(response4.get('total_positive'))
    steam_reviews_dict['neg_reviews'].append(response4.get('total_negative'))
    
steam_reviews_df = pd.DataFrame(steam_reviews_dict)
display(steam_reviews_df)

{'num_reviews': 0, 'review_score': 0, 'review_score_desc': 'No user reviews', 'total_positive': 0, 'total_negative': 0, 'total_reviews': 0}
{'num_reviews': 20, 'review_score': 8, 'review_score_desc': 'Very Positive', 'total_positive': 497, 'total_negative': 62, 'total_reviews': 559}
{'num_reviews': 20, 'review_score': 1, 'review_score_desc': 'Overwhelmingly Negative', 'total_positive': 3864, 'total_negative': 23232, 'total_reviews': 27096}
{'num_reviews': 20, 'review_score': 5, 'review_score_desc': 'Mixed', 'total_positive': 639, 'total_negative': 492, 'total_reviews': 1131}
{'num_reviews': 17, 'review_score': 8, 'review_score_desc': 'Very Positive', 'total_positive': 1590, 'total_negative': 370, 'total_reviews': 1960}
{'num_reviews': 3, 'review_score': 8, 'review_score_desc': 'Very Positive', 'total_positive': 70, 'total_negative': 2, 'total_reviews': 72}
{'num_reviews': 20, 'review_score': 9, 'review_score_desc': 'Overwhelmingly Positive', 'total_positive': 2471, 'total_negative': 75

,appid,total_reviews,pos_reviews,neg_reviews
0,1824220,0,0,0
1,1716120,559,497,62
2,1665460,27096,3864,23232
3,1659040,1131,639,492
4,1651960,1960,1590,370
...,...,...,...,...
274,1623390,40,27,13
275,921800,2994,2426,568
276,1732190,3655,2819,836
277,962700,55,45,10


In [66]:
# pull up missing values in column 'total_reviews'
steam_reviews_df.loc[steam_reviews_df['total_reviews']==0]
# I checked each appid and these games are not yet available on Steam, and thus should be removed

,appid,total_reviews,pos_reviews,neg_reviews


In [67]:
# select all rows where column total_reviews != 0
steam_reviews_df = steam_reviews_df[steam_reviews_df.total_reviews != 0]
display(steam_reviews_df)

,appid,total_reviews,pos_reviews,neg_reviews
1,1716120,559,497,62
2,1665460,27096,3864,23232
3,1659040,1131,639,492
4,1651960,1960,1590,370
5,1647730,72,70,2
...,...,...,...,...
274,1623390,40,27,13
275,921800,2994,2426,568
276,1732190,3655,2819,836
277,962700,55,45,10


In [68]:
#save to_csv()
steam_reviews_df.to_csv('steam_reviews_df.csv', index=False)

##### &emsp;Merge

In [24]:
# read in dataframes to merge and visualize
appid_reviews_mod = pd.read_csv('appid_reviews_mod.csv')
display(appid_reviews_mod)
steam_reviews_df = pd.read_csv('steam_reviews_df.csv')
display(steam_reviews_df)

,appid,name,key_0,game_name,rank,meta_score,user_score,critic_url,4Players.de,Adventure Gamers,...,Wccftech,We Got This Covered,WellPlayed,Windows Central,Worth Playing,XGN,ZTGD,null_404,parsed_name_x,parsed_name_y
0,1824220,Chivalry 2,chivalry2,Chivalry 2,45,82,5,/game/pc/chivalry-2/critic-reviews,83.0,NaN,...,NaN,NaN,NaN,90.0,83.0,NaN,NaN,NaN,Chivalry2,Chivalry2
1,1716120,Monster Rancher 1 & 2 DX,monsterrancher1&2dx,Monster Rancher 1 & 2 DX,294,61,7.5,/game/pc/monster-rancher-1-2-dx/critic-reviews,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MonsterRancher1&2DX,MonsterRancher1&2DX
2,1665460,eFootball™ 2022,efootball2022,eFootball 2022,320,25,1.4,/game/pc/efootball-2022/critic-reviews,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eFootball2022,eFootball2022
3,1659040,HITMAN 3,hitman3,Hitman 3,8,87,7.4,/game/pc/hitman-3/critic-reviews,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,85.0,NaN,Hitman3,HITMAN3
4,1651960,Hextech Mayhem: A League of Legends Story™,hextechmayhemaleagueoflegendsstory,Hextech Mayhem: A League of Legends Story,206,72,7.3,/game/pc/hextech-mayhem-a-league-of-legends-st...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HextechMayhemALeagueofLegendsStory,HextechMayhemALeagueofLegendsStory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,1623390,NaN,unplugged(vertigogames),Unplugged (Vertigo Games),179,74,tbd,/game/pc/unplugged-vertigo-games/critic-reviews,NaN,NaN,...,NaN,NaN,NaN,NaN,67.0,NaN,NaN,NaN,Unplugged(VertigoGames),NaN
275,921800,NaN,encasedascifipostapocalypticrpg,Encased: a sci-fi post-apocalyptic RPG,195,73,7.7,/game/pc/encased-a-sci-fi-post-apocalyptic-rpg...,NaN,NaN,...,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EncasedascifipostapocalypticRPG,NaN
276,1732190,NaN,fatalframemaidenofblackwater,Fatal Frame: Maiden of Black Water,236,69,5,/game/pc/fatal-frame-maiden-of-black-water/cri...,NaN,NaN,...,60.0,NaN,NaN,70.0,NaN,NaN,NaN,NaN,FatalFrameMaidenofBlackWater,NaN
277,962700,NaN,chronicleofinnsmouthmountainsofmadness,Chronicle of Innsmouth: Mountains of Madness,252,68,7.6,/game/pc/chronicle-of-innsmouth-mountains-of-m...,NaN,60.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ChronicleofInnsmouthMountainsofMadness,NaN


,appid,total_reviews,pos_reviews,neg_reviews
0,1716120,559,497,62
1,1665460,27096,3864,23232
2,1659040,1131,639,492
3,1651960,1960,1590,370
4,1647730,72,70,2
...,...,...,...,...
271,1623390,40,27,13
272,921800,2994,2426,568
273,1732190,3655,2819,836
274,962700,55,45,10


In [25]:
df = pd.merge(steam_reviews_df, appid_reviews_mod, how='left', on='appid')
display(df)

#print out each column name
cols = list(df.columns.values)
print(cols)

,appid,total_reviews,pos_reviews,neg_reviews,name,key_0,game_name,rank,meta_score,user_score,...,Wccftech,We Got This Covered,WellPlayed,Windows Central,Worth Playing,XGN,ZTGD,null_404,parsed_name_x,parsed_name_y
0,1716120,559,497,62,Monster Rancher 1 & 2 DX,monsterrancher1&2dx,Monster Rancher 1 & 2 DX,294,61,7.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MonsterRancher1&2DX,MonsterRancher1&2DX
1,1665460,27096,3864,23232,eFootball™ 2022,efootball2022,eFootball 2022,320,25,1.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eFootball2022,eFootball2022
2,1659040,1131,639,492,HITMAN 3,hitman3,Hitman 3,8,87,7.4,...,NaN,NaN,NaN,NaN,NaN,NaN,85.0,NaN,Hitman3,HITMAN3
3,1651960,1960,1590,370,Hextech Mayhem: A League of Legends Story™,hextechmayhemaleagueoflegendsstory,Hextech Mayhem: A League of Legends Story,206,72,7.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HextechMayhemALeagueofLegendsStory,HextechMayhemALeagueofLegendsStory
4,1647730,72,70,2,Zool Redimensioned,zoolredimensioned,Zool Redimensioned,275,65,7.6,...,NaN,NaN,NaN,NaN,70.0,NaN,NaN,NaN,ZoolRedimensioned,ZoolRedimensioned
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,1623390,40,27,13,NaN,unplugged(vertigogames),Unplugged (Vertigo Games),179,74,tbd,...,NaN,NaN,NaN,NaN,67.0,NaN,NaN,NaN,Unplugged(VertigoGames),NaN
272,921800,2994,2426,568,NaN,encasedascifipostapocalypticrpg,Encased: a sci-fi post-apocalyptic RPG,195,73,7.7,...,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EncasedascifipostapocalypticRPG,NaN
273,1732190,3655,2819,836,NaN,fatalframemaidenofblackwater,Fatal Frame: Maiden of Black Water,236,69,5,...,60.0,NaN,NaN,70.0,NaN,NaN,NaN,NaN,FatalFrameMaidenofBlackWater,NaN
274,962700,55,45,10,NaN,chronicleofinnsmouthmountainsofmadness,Chronicle of Innsmouth: Mountains of Madness,252,68,7.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ChronicleofInnsmouthMountainsofMadness,NaN


['appid', 'total_reviews', 'pos_reviews', 'neg_reviews', 'name', 'key_0', 'game_name', 'rank', 'meta_score', 'user_score', 'critic_url', '4Players.de', 'Adventure Gamers', 'Android Central', 'App Trigger', 'Areajugones', 'Atomix', 'Attack of the Fanboy', 'AusGamers', 'BaziCenter', 'Buried Treasure', 'But Why Tho?', 'CD-Action', 'CGMagazine', 'COGconnected', 'Carole Quintaine', 'Checkpoint Gaming', 'Comicbook.com', 'Critical Hit', 'Cubed3', 'Cultured Vultures', 'DarkStation', 'DarkZero', 'Destructoid', 'Dexerto', 'Digital Chumps', 'Digital Spy', 'Digital Trends', 'Digitally Downloaded', 'DualShockers', 'EGM', 'Easy Allies', 'Edge Magazine', 'Eurogamer Italy', 'Everyeye.it', 'Finger Guns', 'Forbes', 'GAMES.CH', 'GAMINGbible', 'GMW3', 'GRYOnline.pl', 'Game Debate', 'Game Informer', 'Game Over Online', 'Game Rant', 'Game Revolution', 'Game World Navigator Magazine', 'GameByte', 'GameCrate', 'GameCritics', 'GameGrin', 'GameMAG', 'GameOver.gr', 'GameSkinny', 'GameSpace', 'GameSpew', 'GameSpo

In [26]:
#rename columns and move to front
col = df.pop('game_name')
df.insert(0, 'game_name', col)

df.pop('name')
df.pop('key_0')

display(df)


,game_name,appid,total_reviews,pos_reviews,neg_reviews,rank,meta_score,user_score,critic_url,4Players.de,...,Wccftech,We Got This Covered,WellPlayed,Windows Central,Worth Playing,XGN,ZTGD,null_404,parsed_name_x,parsed_name_y
0,Monster Rancher 1 & 2 DX,1716120,559,497,62,294,61,7.5,/game/pc/monster-rancher-1-2-dx/critic-reviews,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MonsterRancher1&2DX,MonsterRancher1&2DX
1,eFootball 2022,1665460,27096,3864,23232,320,25,1.4,/game/pc/efootball-2022/critic-reviews,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eFootball2022,eFootball2022
2,Hitman 3,1659040,1131,639,492,8,87,7.4,/game/pc/hitman-3/critic-reviews,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,85.0,NaN,Hitman3,HITMAN3
3,Hextech Mayhem: A League of Legends Story,1651960,1960,1590,370,206,72,7.3,/game/pc/hextech-mayhem-a-league-of-legends-st...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HextechMayhemALeagueofLegendsStory,HextechMayhemALeagueofLegendsStory
4,Zool Redimensioned,1647730,72,70,2,275,65,7.6,/game/pc/zool-redimensioned/critic-reviews,NaN,...,NaN,NaN,NaN,NaN,70.0,NaN,NaN,NaN,ZoolRedimensioned,ZoolRedimensioned
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,Unplugged (Vertigo Games),1623390,40,27,13,179,74,tbd,/game/pc/unplugged-vertigo-games/critic-reviews,NaN,...,NaN,NaN,NaN,NaN,67.0,NaN,NaN,NaN,Unplugged(VertigoGames),NaN
272,Encased: a sci-fi post-apocalyptic RPG,921800,2994,2426,568,195,73,7.7,/game/pc/encased-a-sci-fi-post-apocalyptic-rpg...,NaN,...,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EncasedascifipostapocalypticRPG,NaN
273,Fatal Frame: Maiden of Black Water,1732190,3655,2819,836,236,69,5,/game/pc/fatal-frame-maiden-of-black-water/cri...,NaN,...,60.0,NaN,NaN,70.0,NaN,NaN,NaN,NaN,FatalFrameMaidenofBlackWater,NaN
274,Chronicle of Innsmouth: Mountains of Madness,962700,55,45,10,252,68,7.6,/game/pc/chronicle-of-innsmouth-mountains-of-m...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ChronicleofInnsmouthMountainsofMadness,NaN


In [27]:
df.to_csv('Metacritic_Steam_2021.csv', index=False)